<a href="http://landlab.github.io"><img style="float: left" src="../../../landlab_header.png"></a>

# Stream power, linear diffusion, channel steepness, and relief 
This notebook was created by Nicole Gasparini at Tulane University.

<hr>
<small>For tutorials on learning Landlab, click here: <a href="https://github.com/landlab/landlab/wiki/Tutorials">https://github.com/landlab/landlab/wiki/Tutorials</a></small>
<hr>

**What is this notebook?**

This notebook illustrates the evolution of detachment-limited channels in an actively uplifting landscape. The landscape evolves according to the equation:

\begin{equation}
 \frac{d z}{d t} = U - K_\text{sp} A^{m_{sp}} S^{n_{sp}} + \nabla q_s
\end{equation}

Here the first term on the right hand side is rock uplift, the second term is fluvial incision, and the third term in erosion/deposition due to hillslope processes.

In the fluvial incision term, $K_{sp}$ is the erodibility coefficient, which is thought to be positively correlated with climate wetness, or storminess (this is hard to quantify) and to be negatively correlated with rock strength (again, rock strength is hard to quantify). $m_{sp}$ and $n_{sp}$ are positive exponents, usually thought to have a ratio, $m_{sp}/n_{sp} \approx 0.5$. $A$ is drainage area and $S$ is the slope of steepest descent ($-\frac{dz}{dx}$) where $x$ is horizontal distance (positive in the downslope direction) and $z$ is elevation. (If slope is negative there is no fluvial erosion.) The fluvial erosion term is also known as the stream power equation.

The hillslope sediment transport equation here is described using linear diffusion :
\begin{equation}
q_s = -D \nabla z
\end{equation}
where ${q}_s$ is the transport rate with dimensions of L$^2$T$^{-1}$;  $D$ is a transport coefficient (or diffusivity) with dimensions of L$^2$T$^{-1}$; and $z$ is elevation. $\nabla z$ is the gradient in elevation. If distance is increasing downslope, $\nabla z$ is negative downslope, hence the negative in front of $D$.

You should know the term **drainage density** ($Den$) for this exercise.
\begin{equation}
Den = \frac{\sum l}{A}
\end{equation}

Where $l$ is a length of channel, and $\sum l$ is the length of all channels in the watershed of interest, and $A$ is the drainage area of the watershed of interest. In other words, more length of channels over the same area means a higher drainage density (that should make sense!). 

To calculate drainage density (which we won't do here), one would need to know where the transition from hillslope to channel happens on a landscape. This is oddly hard. One tool that can be used to determine where channels begin is the slope-area plot. At the smallest drainage area are hillslopes. As drainage area increases on a hillslope (moving down the hillslope), the slope increases. This is seen as a positive trend in a slope-area plot. At some drainage area, the slope starts to decrease. At this **critical drainage area**, where the slope-area plot goes from an increasing, positive trend to a decreasing, negative trend, is where the landscape switches from hillslope to channel. The larger the critical drainage area, the longer the hillslopes, the smaller the drainage density.

**What will you do?**

In this exercise you will modify the code to get a better understanding of what sets the relief in a watershed. You will be asked to change the diffusivity and fluvial erodibilty to create watersheds with different drainage densities and explore how relief changes.

Start at the top by reading each block of text and sequentially running each code block (shift - enter OR got to the _Cell_ pulldown menu at the top and choose _Run Cells_). 

If you just change one code block and rerun only that code block, only the parts of the code in that code block will be updated. (E.g. if you change parameters but don't reset the code blocks that initialize run time or topography, then these values will not be reset.) 

**Now on to the code...**

First we have to import the parts of Python and Landlab that are needed to run this code. You should not have to change this first code block.

In [ ]:
# Code block 1

import numpy as np
from matplotlib import pyplot as plt

from landlab import RasterModelGrid, imshow_grid
from landlab.components import (
    ChannelProfiler,
    ChiFinder,
    FlowAccumulator,
    SteepnessFinder,
    StreamPowerEroder,
    LinearDiffuser,
)
from landlab.io import write_esri_ascii

Make a grid and set boundary conditions. 

In [ ]:
# Code Block 2

number_of_rows = 120  # number of raster cells in vertical direction (y)
number_of_columns = 120  # number of raster cells in horizontal direction (x)
dxy = 100  # side length of a raster model cell, or resolution [m]

# Below is a raster (square cells) grid, with equal width and height
mg1 = RasterModelGrid((number_of_rows, number_of_columns), dxy)

Here we make the initial grid of elevation of zeros with a very small amount of noise to make a more pleasing network.

In [ ]:
# Code Block 3

np.random.seed(56)  # seed set so our figures are reproducible
mg1_noise = (np.random.rand(mg1.number_of_nodes) / 1000.0
             )  # intial noise on elevation gri

# set up the elevation on the grid
z1 = mg1.add_zeros("topographic__elevation", at="node")
z1 += mg1_noise

# Set boundary conditions - only the south side of the grid is open.
# Boolean parameters are sent to function in order of
# east, north, west, south.
mg1.set_watershed_boundary_condition("topographic__elevation")

Set parameters related to time.

In [ ]:
# Code Block 4

tmax = 1e6  # time for the model to run [yr] (Original value was 1E6 yr)
dt = 1000  # time step [yr] (Original value was 1000 yr)
total_time = 0  # amount of time the landscape has evolved [yr]
# total_time will increase as you keep running the code.

t = np.arange(0, tmax, dt)  # each of the time steps that the code will run

Set parameters for incision and intializing all of the process components that do the work. We also initialize tools for quantifying the landscape.

In [ ]:
# Code Block 5

# Original K_sp value is 1e-5
K_sp = 1.0e-5  # units vary depending on m_sp and n_sp
m_sp = 0.5  # exponent on drainage area in stream power equation
n_sp = 1.0  # exponent on slope in stream power equation

frr = FlowAccumulator(mg1, flow_director='FlowDirectorD8')  # intializing flow routing
spr = StreamPowerEroder(mg1, K_sp=K_sp, m_sp=m_sp, n_sp=n_sp,
                        threshold_sp=0.0)  # initializing stream power incision

D = 0.5  # initial value of 0.5 m^2/yr
# initialize the linear diffusion component
ld = LinearDiffuser(mg1, linear_diffusivity=D, deposit=False)

theta = m_sp / n_sp
# initialize the component that will calculate channel steepness
sf = SteepnessFinder(mg1, reference_concavity=theta, min_drainage_area=1000.0)
# initialize the component that will calculate the chi index
cf = ChiFinder(mg1,
               min_drainage_area=1000.0,
               reference_concavity=theta,
               use_true_dx=True)

Initialize rock uplift rate. This will need to be changed later.

In [ ]:
# Code Block 6

#  uplift_rate [m/yr] (Original value is 0.0001 m/yr)
uplift_rate = np.ones(mg1.number_of_nodes) * 0.0001

Now for the code loop. 

Note that you can rerun Code Block 7 many times, and as long as you don't reset the elevation field (Code Block 3), it will take the already evolved landscape and evolve it even more. If you want to change parameters in other code blocks (e.g. Code Block 5 or 6), you can do that too, and as long as you don't reset the elevation field (Code Block 3) the new parameters will apply on the already evolved topography. 

In [ ]:
# Code Block 7

for ti in t:
    z1[mg1.
       core_nodes] += uplift_rate[mg1.core_nodes] * dt  # uplift the landscape
    ld.run_one_step(dt) # hillslope evolution
    frr.run_one_step()  # route flow
    spr.run_one_step(dt)  # fluvial incision
    total_time += dt  # update time keeper
    print(total_time)

Plot the topography.

In [ ]:
# Code Block 8

imshow_grid(mg1,
            "topographic__elevation",
            grid_units=("m", "m"),
            var_name="Elevation (m)")
title_text = f"$K_{{sp}}$={K_sp}; $D$={D}; $time$={total_time} yr; $dx$={dxy} m"
plt.title(title_text)

max_elev = np.max(z1)
print("Maximum elevation is ", np.max(z1))

Plot the slope and area data at each point on the landscape (in log-log space). We will only plot the core nodes because the boundary nodes have slopes that are influenced by the boundary conditions. 

In [ ]:
# Code Block 9

plt.loglog(
    mg1.at_node["drainage_area"][mg1.core_nodes],
    mg1.at_node["topographic__steepest_slope"][mg1.core_nodes],
    "b.",
)
plt.ylabel("Topographic slope")
plt.xlabel("Drainage area (m^2)")
title_text = f"$K_{{sp}}$={K_sp}; $D$={D}; $time$={total_time} yr; $dx$={dxy} m"

plt.title(title_text)

It is slightly easier to interpret slope-area data when we look at a single channel, rather than the entire landscape. Below we plot the profile and slope-area data for the longest length of channel on the landscape.

In [ ]:
# Code Block 10

# profile the largest channels, set initially to find the mainstem channel in the three biggest watersheds
# you can change the number of watersheds, or choose to plot all the channel segments in the watershed that
# have drainage area below the threshold (here we have set the threshold to the area of a grid cell).
prf = ChannelProfiler(mg1,
                      number_of_watersheds=1,
                      main_channel_only=True,
                      minimum_channel_threshold=dxy**2)
prf.run_one_step()

# plot the elevation as a function of distance upstream
plt.figure(1)
title_text = f"$K_{{sp}}$={K_sp}; $D$={D}; $time$={total_time} yr; $dx$={dxy} m"
prf.plot_profiles(xlabel='distance upstream (m)',
                  ylabel='elevation (m)',
                  title=title_text)

# plot the location of the channels in map view
plt.figure(2)
prf.plot_profiles_in_map_view()

# slope-area data in just the profiled channels
plt.figure(3)
for i, outlet_id in enumerate(prf.data_structure):
    for j, segment_id in enumerate(prf.data_structure[outlet_id]):
        if j == 0:
            label = "channel {i}".format(i=i + 1)
        else:
            label = '_nolegend_'
        segment = prf.data_structure[outlet_id][segment_id]
        profile_ids = segment["ids"]
        color = segment["color"]
        plt.loglog(
            mg1.at_node["drainage_area"][profile_ids],
            mg1.at_node["topographic__steepest_slope"][profile_ids],
            '.',
            color=color,
            label=label,
        )

plt.legend(loc="lower left")
plt.xlabel("drainage area (m^2)")
plt.ylabel("topographic slope [m/m]")
title_text = f"$K_{{sp}}$={K_sp}; $D$={D}; $time$={total_time} yr; $dx$={dxy} m"
plt.title(title_text)

The chi index is a useful way to quantitatively interpret fluvial channels. Below we plot the chi index in the three largest channels and also a chi map across the entire landscape. 

In [ ]:
# # Code Block 11

# # calculate the chi index
# cf.calculate_chi()

# # chi-elevation plots in the profiled channels
# plt.figure(4)

# for i, outlet_id in enumerate(prf.data_structure):
#     for j, segment_id in enumerate(prf.data_structure[outlet_id]):
#         if j == 0:
#             label = "channel {i}".format(i=i + 1)
#         else:
#             label = '_nolegend_'
#         segment = prf.data_structure[outlet_id][segment_id]
#         profile_ids = segment["ids"]
#         color = segment["color"]
#         plt.plot(
#             mg1.at_node["channel__chi_index"][profile_ids],
#             mg1.at_node["topographic__elevation"][profile_ids],
#             color=color,
#             label=label,
#         )

# plt.xlabel("chi index (m)")
# plt.ylabel("elevation (m)")
# plt.legend(loc="lower right")
# title_text = f"$K_{{sp}}$={K_sp}; $time$={total_time} yr; $dx$={dxy} m; concavity={theta}"
# plt.title(title_text)

# # chi map
# plt.figure(5)
# imshow_grid(
#     mg1,
#     "channel__chi_index",
#     grid_units=("m", "m"),
#     var_name="Chi index (m)",
#     cmap="jet",
# )
# title_text = f"$K_{{sp}}$={K_sp}; $time$={total_time} yr; $dx$={dxy} m; concavity={theta}"
# plt.title(title_text)

The channel steepness index is another useful index to quantify fluvial channels. Below we plot the steepness index in the same three largest channels, and also plot steepness index across the grid.

In [ ]:
# Code Block 12

# calculate channel steepness
sf.calculate_steepnesses()

# plots of steepnes vs. distance upstream in the profiled channels
plt.figure(6)

for i, outlet_id in enumerate(prf.data_structure):
    for j, segment_id in enumerate(prf.data_structure[outlet_id]):
        if j == 0:
            label = "channel {i}".format(i=i + 1)
        else:
            label = '_nolegend_'
        segment = prf.data_structure[outlet_id][segment_id]
        profile_ids = segment["ids"]
        distance_upstream = segment["distances"]
        color = segment["color"]
        plt.plot(
            distance_upstream,
            mg1.at_node["channel__steepness_index"][profile_ids],
            'x',
            color=color,
            label=label,
        )

plt.xlabel("distance upstream (m)")
plt.ylabel("steepness index")
plt.legend(loc="upper left")
plt.title(
    f"$K_{{sp}}$={K_sp}; $D$={D}; $time$={total_time} yr; $dx$={dxy} m; concavity={theta}"
)

# channel steepness map
plt.figure(7)
imshow_grid(
    mg1,
    "channel__steepness_index",
    grid_units=("m", "m"),
    var_name="Steepness index ",
    cmap="jet",
)
title_text = ("$K_{sp}$=" + str(K_sp) + "; $time$=" + str(total_time) +
              "yr; $dx$=" + str(dxy) + "m" + "; concavity=" + str(theta))
plt.title(
    f"$K_{{sp}}$={K_sp}; $D$={D}; $time$={total_time} yr; $dx$={dxy} m; concavity={theta}"
)

If you have a grid that you want to export, uncomment and edit the appropriate lines below and run the code block.

In [ ]:
# Code Block 13

## Below has the name of the file that data will be written to.
## You need to change the name of the file every time that you want
## to write data, otherwise you will get an error.
## This will write to the directory that you are running the code in.
# write_file_name = 'data_file.txt'
## Below is writing elevation data in the ESRI ascii format so that it can
## easily be read into Arc GIS or back into Landlab.
# write_esri_ascii(write_file_name, mg1, 'topographic__elevation')

After running every code block once, has the landscape reached steady state? Answer: NO! How do you know? After you think about this, you are ready to complete this project.

Answer the following questions using the code above and below. All answers and supporting figures (produced using the code) should be embedded in a powerpoint that you hand in. Code Blocks 8-12 produce different figures that you may find useful. You can use any or all of these different figures to help you with the questions below. (Download or screenshoot the figures.) 

Please complete the following tasks. Make sure your write in full sentences and proofread the document that you hand in.

1. Using the parameters provided in the initial notebook, run the landscape to steady state. (Note that you can keep running the main evolution loop - Code Block 7 - and the different plotting blocks without running the code blocks above them.) [These landscapes may not reach a perfect steady state. Close is fine.] When the landscape reaches steady state, record the channel steepness, $k_{sn}$ (only applies for the channelized part of the landscape) and total relief (in this case the minimum elevation is zero, so the total relief is just the maximum elevation). Also note the approximate critical drainage area. (See the description at the top of this notebook if you don't know that term.) Save some illustrative plots. This example has a relatively small critical drainage area/large drainage density. (10 pts)

2. Rerun the notebook with a new diffusivity value, but keep all other parameters the same (so do not change the fluvial erodibility or rock uplift value). You should find a scenario with a critical drainage area higher than the initial value, but less than 1e6 m$^2$. Run to steady state, and record the critical drainage area, channel steepness index, and total relief. Make sure you also record the diffusivity value used. Save some illustrative plots. (10 pts)

3. Repeat step 2. So same everything except use a third, different diffusivity value. This example should have a different critical drainage from the previous two landscapes, and the critical drainage area should be less than 1e6 m$^2$. (10 pts)

3. Now repeat steps 1, 2, and 3, two more times. I want you to choose two different fluvial erodibility values (both should be smaller than the original value of $K_{sp} = 1E-5$, i.e. they should produce larger $k_{sn}$ values). For each $K_{sp}$ value, you should produce three landscapes with three different diffusivity values. Try to keep all slope values (on hillslopes and channels) less than 1 m/m. Try to keep the critical drainage area less than 1e6 m$^2$. Record critical drainage area, channel steepness index, and total relief for all the landscapes. (20 pts)

4. You should have produced a total of 9 steady-state landscapes, using three different fluvial erodibility values, and at least three different diffusivity values. In all cases, the uplift rate remained the same, so the erosion rate remained the same. What trends do you notice about controls on relief and channel steepness index? Is relief related to critical drainage area? Is it related to channel steepness index? Can you summarize (in 5 sentences or less) some things you have learned about rivers and fluvial erodbility, hillslopes and diffusivity, and topographic trends? (15 pts)

5. Thought question ... If you had a larger watershed, do you think that would impact any of the trends (or lack of trends) that you observed? Please explain in 5 sentences or less. (5 pts)